In [1]:
import pandas as pd
from sklearn import tree
import ast
import math
import pickle
import os
import glob

## Use the same config as in the later game

In [21]:
#[General_Constants]
FIELD_WIDTH = 19
FIELD_HEIGHT = 15

#[Game_Constants]
NO_ITERATIONS = 100
MAX_CALC_TIME = 100000

#[Field_Constants]
CELL_EMPTY = '.'
CELL_SHEEP_1 = 'S'
CELL_SHEEP_1_d = 'U'
CELL_WOLF_1 = 'W'
CELL_SHEEP_2 = 's'
CELL_SHEEP_2_d = 'u'
CELL_WOLF_2 = 'w'
CELL_GRASS = 'g'
CELL_RHUBARB = 'r'
CELL_FENCE = '#'


#[Movements]
MOVE_NONE = 0
MOVE_UP = -1
MOVE_DOWN = 1
MOVE_LEFT = -2
MOVE_RIGHT = 2

#[Awards]
AWARD_RHUBARB = 5
AWARD_GRASS = 1

# Load data

Load all games into one dataframe

In [2]:
# path = "../example_player/training_data" 
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0, keep_default_na=False)    
#     df['game'] = filename
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)

In [23]:
path = "../example_player/training_data/subset" 
all_files = glob.glob(path + "/*.csv")   

training_data = []

#load the data into a pandas frames
for file in all_files:
    game_data = pd.read_csv(file,index_col=False)
    reason = game_data.iloc[-1][6]
    
    #if the reason is found, add it to each line to fill out the blanks
    if type(reason) is str:
        for index,row in game_data.iterrows():
            game_data.loc[index,'reason'] = reason

    #else there was no reason, implying the game reached the number of iterations
    else:
        for index,row in game_data.iterrows():
            game_data.loc[index,'reason'] = 'max_iterations'    
    
    training_data.append(game_data)

#preview the final 5 lines
training_data[-1].head()

,field_before,field_after,turn_made_by,move_made,score1,score2,reason
0,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,2,0,0,sheep1 eaten
1,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player2 sheep,-1,0,0,sheep1 eaten
2,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,2,0,0,sheep1 eaten
3,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player2 sheep,-1,0,0,sheep1 eaten
4,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 wolf,2,0,0,sheep1 eaten


## Drop unneeded rows

Use only where:
* shows winner movement
* ending reason is '' (ended because of max iterations) or sheep eaten

In [167]:
# df_train = frame[((frame.reason == 'sheep1 eaten') 
#                   | (frame.reason == 'sheep2 eaten')
#                   | (frame.reason == ''))]

# df_train = df_train.join(df_train.groupby('game').agg({'score1':'max','score2':'max'}),on='game',rsuffix='_max')

In [168]:
# def is_winner(row):
#     if '1' in row.turn_made_by:
#         comp= row['score1_max'] > row['score2_max']
#     else:
#         comp= row['score2_max'] > row['score1_max']
#     return comp

# df_train = df_train[df_train.apply(lambda row: is_winner(row),axis=1)]

## Generate general use features

position of wolf, sheep, closest food

In [169]:
# sm = df_train[:10]

In [170]:
# def get_pos(row):    
#     y = 0
#     for r in ast.literal_eval(row['field_before']):
#         x = 0
#         for item in r:
#             if item == 'W':
#                 w1_x = x
#                 w1_y = y
#             elif item == 'S':
#                 s1_x = x
#                 s1_y = y
#             elif item == 'w':
#                 w2_x = x
#                 w2_y = y
#             elif item == 's':
#                 s2_x = x
#                 s2_y = y
#             x += 1
#         y += 1
#     return (w1_x,w1_y,s1_x,s1_y,w2_x,w2_y,s2_x,s2_y)

# sm['pos_'] = sm.apply(lambda row: get_pos(row), axis=1)

/home/chris/PAI/Ex2/venv/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [171]:
# sm[['pos_w1_x', 'pos_w1_y', 'pos_s1_x', 'pos_s1_y', 'pos_w2_x', 'pos_w2_y', 'pos_s2_x', 'pos_s2_y']] = pd.DataFrame(sm['pos_'].tolist(), index=sm.index)  

# Feature selection and Instance selection 

# TODOs

1. PCA to see which feature has how much influence
2. check which moves would be allowed

In [22]:
training_data

[                                          field_before  \
 0    [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 1    [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 2    [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 3    [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 4    [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 5    [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 6    [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 7    [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 8    [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 9    [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 10   [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 11   [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 12   [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 13   [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 14   [['.', '.', '.', '.', '.', '.', '.', '.', '.',...   
 15   [['.', '.', '.', '.', '.', '.', '.', '.', '.',... 

## Sheep

calculate x and y distance to wolf and to closest food


In [29]:
 def valid_move(sheep_x, sheep_y, new_x, new_y, field, player, is_sheep = True):
    # Neither the sheep nor the wolf, can step on a square outside the map. Imagine the map is surrounded by fences.
    if new_x > FIELD_HEIGHT - 1:
        return False
    elif new_x < 0:
        return False
    elif new_y > FIELD_WIDTH - 1:
        return False
    elif new_y < 0:
        return False

    # Neither the sheep nor the wolf, can enter a square with a fence on.
    if field[new_x][new_y] == CELL_FENCE:
        return False

    if(is_sheep):
        # Sheep can not step on squares occupied by the wolf of the same player.
        # Sheep can not step on squares occupied by the opposite sheep.
        if '1' in player:
            if field[new_x][new_y] == CELL_SHEEP_2 or \
                    field[new_x][new_y] == CELL_WOLF_1 or \
                    field[new_x][new_y] == CELL_WOLF_2:
                return False
        else:
            if field[new_x][new_y] == CELL_SHEEP_1 or \
                    field[new_x][new_y] == CELL_WOLF_2 or \
                    field[new_x][new_y] == CELL_WOLF_1:
                return False
    else:
        # Wolfs can not step on squares occupied by the opponents wolf (wolfs block each other).
        # Wolfs can not step on squares occupied by the sheep of the same player .
        if '1' in player:
            if field[new_x][new_y] == CELL_WOLF_2:
                return False
            elif field[new_x][new_y] == CELL_SHEEP_1:
                return False
        elif figure == CELL_WOLF_2:
            if field[new_x][new_y] == CELL_WOLF_1:
                return False
            elif field[new_x][new_y] == CELL_SHEEP_2:
                return False
    return True

In [32]:
X_sheep = []
Y_sheep = []
number_moves = 0

for game in training_data:
    
    #we want to learn from the winning player, which is the player with the highest score:
    if game.iloc[-1][4] < game.iloc[-1][5]:
        sheep = 's'
        wolf = 'W'
    
    elif game.iloc[-1][4] > game.iloc[-1][5]:
        sheep = 'S'
        wolf = 'w'
    else:
        continue

    rhubarb = 'r'
    grass = 'g'

    #for each game state in our training data
    for index,row in game.iterrows():

        #we don't want games that ended because of an error or because the sheep commited suicide
        if row['reason'] not in ('sheep1 eaten','sheep2 eaten','max_iterations'):
            continue

        #we want to only learn from sheep
        if row['move_made'] == 'player1 wolf' or row['move_made'] == 'player2 wolf':
            continue
        
        number_moves += 1
        
        #this is the move that we are learning from this game state
        move = row['move_made']

        #create empty feature array for this game state
        game_features = []

        #turn the field from before the move from a string back to a list
        field= ast.literal_eval(row['field_before'])

        #get positions of sheep, wolf and food items
        food = []
        y=0
        for field_row in field:
            x = 0
            for item in field_row:
                if item == sheep:
                    sheep_position = (x, y)
                elif item == wolf:
                    wolf_position = (x, y)
                elif item == rhubarb or item == grass:
                    food.append((x, y))
                x += 1
            y += 1

        # feature 1: x-distance wolf
        sf_x_wolf = sheep_position[0] - wolf_position[0]
        # feature 2: y-distance wolf
        sf_y_wolf = sheep_position[1] - wolf_position[1]

        # determine closest food:
        food_distance = 1000
        food_goal = None
        for food_item in food:
            distance = abs(food_item[0] - sheep_position[0]) + abs(food_item[1] - sheep_position[1])
            if distance < food_distance:
                food_distance = distance
                food_goal = food_item

        # feature 3: x-distance to food
        sf_x_food = 0
        # feature 4: y-distance to food
        sf_y_food = 0               
        if food_goal:
            sf_x_food = sheep_position[0] - food_goal[0]
            sf_y_food = sheep_position[1] - food_goal[1]

        # feature 5: is going up allowed?         
        sf_up = valid_move(sheep_position[0], sheep_position[1], sheep_position[0]-1, sheep_position[1], field,row['turn_made_by'],True)
        # feature 6: is going down allowed?         
        sf_down = valid_move(sheep_position[0], sheep_position[1], sheep_position[0]+1, sheep_position[1], field,row['turn_made_by'],True)
        # feature 7: is going right allowed?         
        sf_right = valid_move(sheep_position[0], sheep_position[1]+1, sheep_position[0], sheep_position[1], field,row['turn_made_by'],True)
        # feature 8: is going left allowed?         
        sf_left = valid_move(sheep_position[0], sheep_position[1]-1, sheep_position[0], sheep_position[1], field,row['turn_made_by'],True)
                      
        game_features.append(sf_x_wolf)
        game_features.append(sf_y_wolf)
        game_features.append(sf_x_food)
        game_features.append(sf_y_food)
        game_features.append(sf_up)
        game_features.append(sf_down)
        game_features.append(sf_right)
        game_features.append(sf_left)

        #add features and move to X_sheep and Y_sheep
        X_sheep.append(game_features)
        Y_sheep.append(move)

#this prints an example of our feature and outcome vector:
print(X_sheep[0])
print(Y_sheep[0])
print(number_moves)

[-14, -12, -6, -5, True, True, True, True]
2
2992


In [6]:
len(X_sheep)

2992

In [115]:
# # we want to only learn from sheep
# df_sheep = df_train[((df_train.turn_made_by == 'player1 sheep') | (df_train.turn_made_by == 'player2 sheep'))]

# df_sheep['f1'] = 

,field_before,field_after,turn_made_by,move_made,score1,score2,reason,game,score1_max,score2_max,is_winner
0,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,1,0,0,,../example_player/training_data/2029.csv,5,2,True
2,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,1,0,0,,../example_player/training_data/2029.csv,5,2,True
6,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,1,0,0,,../example_player/training_data/2029.csv,5,2,True
8,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,1,0,0,,../example_player/training_data/2029.csv,5,2,True
12,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,1,0,0,,../example_player/training_data/2029.csv,5,2,True
14,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,1,0,0,,../example_player/training_data/2029.csv,5,2,True
18,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,2,0,0,,../example_player/training_data/2029.csv,5,2,True
20,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,2,0,0,,../example_player/training_data/2029.csv,5,2,True
24,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,2,0,0,,../example_player/training_data/2029.csv,5,2,True
26,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,2,0,0,,../example_player/training_data/2029.csv,5,2,True


In [15]:

# X_sheep = []
# Y_sheep = []
# number_moves = 0

# for game in training_data:

#     # we want to learn from the winning player, which is the player with the highest score:
#     if game.iloc[-1][4] < game.iloc[-1][5]:
#         sheep = 's'
#         wolf = 'W'

#     elif game.iloc[-1][4] > game.iloc[-1][5]:
#         sheep = 'S'
#         wolf = 'w'
#     else:
#         continue

#     rhubarb = 'r'
#     grass = 'g'

#     # for each game state in our training data
#     for index, row in game.iterrows():

#         # we don't want games that ended because of an error or because the sheep commited suicide
#         if row['reason'] not in ('sheep1 eaten', 'sheep2 eaten', 'max_iterations'):
#             continue

#         # we want to only learn from sheep
#         if row['move_made'] == 'player1 wolf' or row['move_made'] == 'player2 wolf':
#             continue

#         number_moves += 1

#         # this is the move that we are learning from this game state
#         move = row['move_made']

#         # create empty feature array for this game state
#         game_features = []

#         # turn the field from before the move from a string back to a list
#         field = ast.literal_eval(row['field_before'])

#         # get positions of sheep, wolf and food items
#         food = []
#         y = 0
#         for field_row in field:
#             x = 0
#             for item in field_row:
#                 if item == sheep:
#                     sheep_position = (x, y)
#                 elif item == wolf:
#                     wolf_position = (x, y)
#                 elif item == rhubarb or item == grass:
#                     food.append((x, y))
#                 x += 1
#             y += 1
#         # feature 1: x-distance wolf
#         s_feature1 = sheep_position[0] - wolf_position[0]
#         # feature 2: y-distance wolf
#         s_feature2 = sheep_position[1] - wolf_position[1]

       
#         # determine closest food:
#         food_distance = 1000
#         food_goal = None
#         for food_item in food:
#             distance = abs(food_item[0] - sheep_position[0]) + abs(food_item[1] - sheep_position[1])
#             if distance < food_distance:
#                 food_distance = distance
#                 food_goal = food_item

#         # feature 3: x-distance to food
#         s_feature3 = 0
#         # feature 4: y-distance to food
#         s_feature4 = 0

#         if food_goal:
#             s_feature3 = sheep_position[0] - food_goal[0]
#             s_feature4 = sheep_position[1] - food_goal[1]

#     game_features.append(s_feature1)
#     game_features.append(s_feature2)
#     game_features.append(s_feature3)
#     game_features.append(s_feature4)

#     # add features and move to X_sheep and Y_sheep
#     X_sheep.append(game_features)
#     Y_sheep.append(move)

this prints an example of our feature and outcome vector:

In [16]:
# print(X_sheep[0])
# print(Y_sheep[0])
# print(number_moves)

[1, 2, 0, 0]
1
229206


## Wolf

Explain here in text which feature you used for the wolf, and which data you used for training.

In [17]:
#construct your features here

# Train sheep

In [18]:
sheep_tree = tree.DecisionTreeClassifier()
sheep_tree = sheep_tree.fit(X_sheep,Y_sheep)

ValueError: setting an array element with a sequence.

# Train wolf

In [ ]:
# wolf_tree = tree.DecisionTreeClassifier()
# wolf_tree = wolf_tree.fit(X_wolf,Y_wolf)

# Save models to files

Save your models to files here using pickle. Change the [uzhshortname] to your own UZH shortname. This name needs to match the model that you caller in your python player file.

In [ ]:
sheep_filename = 'chriweb_sheep_model.sav'
# wolf_filename = 'chriweb_wolf_model.sav'

pickle.dump(sheep_tree,open(sheep_filename,'wb'))
# pickle.dump(wolf_tree,open(wolf_filename,'wb'))